In [2]:
import numpy as np
import math

In [126]:
def mod2(A):
    """Reduce a numpy integer matrix A modulo 2."""
    return np.remainder(A, 2)

def build_2d_matrices(delta0):
    k, j = delta0.shape
    I_k = np.eye(k, dtype=int)  # Identity matching # of rows of H
    I_j = np.eye(j, dtype=int)  # Identity matching # of columns of H

    # Eq. (43):
    #  tilde_delta_{-1} = [ 1 ⊗ delta0^T
    #                      delta0 ⊗ 1    ]
    delta_m1_2d = np.block([
        [np.kron(I_j, delta0.T)],
        [np.kron(delta0, I_k)]
    ])
    
    #  tilde_delta_0 = [ delta0 ⊗ 1   |   1 ⊗ delta0^T ]
    delta_0_2d = np.block([
        [np.kron(delta0, I_j), np.kron(I_k, delta0.T)]
    ])

    return mod2(delta_m1_2d), mod2(delta_0_2d)

def build_4d_matrices(delta_m1_2d, delta_0_2d):
    """
    Parameters
    ----------
    delta_m1_2d : The matrix δ₋₁ from previous construction.
    delta_0_2d : The matrix δ₀ from previous construction.

    Returns
    -------
    mz : Eqn. (58)
    hz : Eqn. (59)
    hx : Eqn. (60)
    mx : Eqn. (61) 
    """
    k, j = delta_0_2d.shape

    # (58) δ₋₂ =  ( I ⊗ δ₀ᵀ )
    #             ( δ₋₁ ⊗ I  )
    mz = np.vstack([
        np.kron(np.eye(k), delta_0_2d.T),
        np.kron(delta_m1_2d, np.eye(k))
    ])

    # (59) δ₋₁(new):
    top_left  = np.kron(np.eye(k), delta_m1_2d.T)
    top_right = np.zeros_like(top_left)
    mid_left  = np.kron(delta_m1_2d, np.eye(j))
    mid_right = np.kron(np.eye(j), delta_0_2d.T)
    bot_right = np.kron(delta_0_2d, np.eye(k))
    bot_left = np.zeros_like(bot_right)
    hz = np.block([
        [top_left,  top_right],
        [mid_left,  mid_right],
        [bot_left,  bot_right]
    ])

    # (60) δ₀(new) :
    top_left  = np.kron(delta_m1_2d, np.eye(k))
    top_mid = np.kron(np.eye(j), delta_m1_2d.T)
    top_right = np.zeros_like(top_left)
    bot_left  = np.zeros_like(top_left)
    bot_mid = np.kron(delta_0_2d, np.eye(j))
    bot_right = np.kron(np.eye(k), delta_0_2d.T)
    hx = np.block([
        [top_left, top_mid, top_right],
        [bot_left,  bot_mid, bot_right]
    ])

    # (61) δ₁(new):
    mx =  np.hstack([
        np.kron(delta_0_2d, np.eye(k)),
        np.kron(np.eye(k), delta_m1_2d.T)
    ])

    return mz, hz, hx, mx

In [68]:
def verify_delta_condition(delta_i, delta_i_minus_1):
    """
    Verify that δ_i * δ_{i-1} = 0 mod 2.
    """
    product = np.mod(np.dot(delta_i, delta_i_minus_1), 2)  # Compute δ_i * δ_{i-1} mod 2
    return np.all(product == 0)  # Check if all entries are zero

delta0_works = np.array([[1, 1, 0], 
                         [0, 1, 1]], dtype=float)


delta0_fails = np.array([[1, 1, 0], 
                           [0, 1, 1,], 
                           [1, 0, 1]], dtype=float)

delta0_try = np.array([[1, 1, 0, 0], 
                       [0, 1, 1, 0], 
                       [0, 0, 1, 1]], dtype=float)

for delta0 in [delta0_works, delta0_fails, delta0_try]:
    dm1, d0 = build_2d_matrices(delta0)
    print(f'input matrix shape: {delta0.shape}')
    mz, hz, hx, mx = build_4d_matrices(dm1, d0)
    print("mz:", mz.shape)
    print("hz", hz.shape,)
    print("hx:", hx.shape)
    print("mx:", mx.shape)

    # Verify condition δ_{i} * δ_{i-1} = 0 mod 2 for each consecutive pair
    print("δ_{-2} * δ_{-1} ≡ 0 mod 2:", verify_delta_condition(hz, mz))
    print("δ_{-1} * δ_{0} ≡ 0 mod 2:", verify_delta_condition(hx, hz))
    print("δ_{0} * δ_{1} ≡ 0 mod 2:", verify_delta_condition(mx, hx))
    print()

input matrix shape: (2, 3)
k, j: 6, 13
mz: (156, 36)
hz (241, 156)
hx: (156, 241)
mx: (36, 156)
δ_{-2} * δ_{-1} ≡ 0 mod 2: True
δ_{-1} * δ_{0} ≡ 0 mod 2: True
δ_{0} * δ_{1} ≡ 0 mod 2: True

input matrix shape: (3, 3)
k, j: 9, 18
mz: (324, 81)
hz (486, 324)
hx: (324, 486)
mx: (81, 324)
δ_{-2} * δ_{-1} ≡ 0 mod 2: True
δ_{-1} * δ_{0} ≡ 0 mod 2: True
δ_{0} * δ_{1} ≡ 0 mod 2: True

input matrix shape: (3, 4)
k, j: 12, 25
mz: (600, 144)
hz (913, 600)
hx: (600, 913)
mx: (144, 600)
δ_{-2} * δ_{-1} ≡ 0 mod 2: True
δ_{-1} * δ_{0} ≡ 0 mod 2: True
δ_{0} * δ_{1} ≡ 0 mod 2: True



In [52]:
import numpy as np
from ldpc.codes import ring_code
from ldpc.code_util import compute_code_distance
import ldpc.mod2 as mod2
from ldpc import protograph as pt
from ldpc.codes import hamming_code
from bposd.css import css_code
from bposd.hgp import hgp
from lifted_hgp import *
from css_ss_decode_sim import css_ss_decode_sim

In [104]:
a1=pt.array([
        [(0), (11), (7), (12)],
        [(1), (8), (1), (8)],
        [(11), (0), (4), (8)],
        [(6), (2), (4), (12)]])

from lifted_hgp import lifted_hgp
qcode=lifted_hgp(lift_parameter=13,a=a1,b=a1)

# hx=qcode.hx_proto.to_binary(lift_parameter=13)
# hz=qcode.hz_proto.to_binary(lift_parameter=13)

# qcode=css_code(hx,hz)
# qcode.test()

In [106]:
a1.to_binary(lift_parameter=13).shape

(52, 52)

In [108]:
hx.shape

(208, 416)

In [110]:
hz.shape

(208, 416)

In [266]:
H=pt.array([[(0,1)]]).to_binary(lift_parameter=4)

dm1, d0 = build_2d_matrices(H)
mz, hz, hx, mx = build_4d_matrices(dm1, d0)

print("δ_{-2} * δ_{-1} ≡ 0 mod 2:", verify_delta_condition(hz, mz))
print("δ_{-1} * δ_{0} ≡ 0 mod 2:", verify_delta_condition(hx, hz))
print("δ_{0} * δ_{1} ≡ 0 mod 2:", verify_delta_condition(mx, hx))

qcode=css_code(hx,hz.T)
qcode.test()

δ_{-2} * δ_{-1} ≡ 0 mod 2: True
δ_{-1} * δ_{0} ≡ 0 mod 2: True
δ_{0} * δ_{1} ≡ 0 mod 2: True
<Unnamed CSS code>
 -Block dimensions: Pass
 -PCMs commute hz@hx.T==0: Pass
 -PCMs commute hx@hz.T==0: Pass
 -lx \in ker{hz} AND lz \in ker{hx}: Pass
 -lx and lz anticommute: Pass
 -<Unnamed CSS code> is a valid CSS code w/ params [1536,6,nan]


True

In [164]:
hx.shape

(1024, 1536)

In [261]:
nx, N = hx.shape
e = np.random.randint(0, 2, size=(N, 1))
f = np.random.randint(0, 2, size=(N, 1))
u_e = np.random.randint(0, 2, size=(nx, 1))
u_f = np.random.randint(0, 2, size=(nx, 1))

# Compute syndromes from hx and hz
syndrome_x = (hx @ e) % 2
syndrome_z = (hz.T @ e) % 2

# Compute syndromes from mx and mz
measured_syndrome_x = (mx @ u_e) % 2
measured_syndrome_z = (mz.T @ u_f) % 2